# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
file = "output_data/cities.csv"

cities_df = pd.read_csv(file)

cities_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Eyl,SO,7.98,49.82,83.08,72,0,7.07,1588974135
1,Vaini,TO,-21.20,-175.20,80.60,88,75,11.41,1588974135
2,Boyolangu,ID,-8.12,111.89,76.80,90,97,1.61,1588974135
3,Mar del Plata,AR,-38.00,-57.56,59.00,72,49,12.75,1588973959
4,Coahuayana Viejo,MX,18.73,-103.68,89.60,43,40,13.87,1588974136
...,...,...,...,...,...,...,...,...,...
580,Mehrān,IR,33.12,46.16,59.00,44,0,6.93,1588974283
581,Charcas,MX,23.13,-101.12,78.12,23,77,7.76,1588974283
582,Orotukan,RU,62.27,151.70,27.28,95,95,2.24,1588974283
583,Kangalassy,RU,62.35,129.96,28.40,86,75,6.71,1588974283


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)
humidity = cities_df['Humidity'].astype(float)
locations = cities_df[['Lat','Lng']]

In [26]:
fig = gmaps.figure(center = [2,5], zoom_level =1.9)

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, dissipating = False, point_radius=3)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
# Create a temperature range suitable for me from original DataFrame
temp_df = cities_df[(cities_df['Max Temp'] >= 75) & (cities_df['Max Temp'] <=85)]

# Create a maximum wind range suitable for me from original DataFrame
temp_df = temp_df[temp_df['Wind Speed'] <= 20]

# Create a threshold of clouds suitable for me from original DataFrame
temp_df = temp_df[temp_df['Cloudiness'] <= 10]

# Create a threshold of humidity suitable for me from original DataFrame
temp_df = temp_df[temp_df['Humidity'] <= 35]

# Clear out any not a number values
temp_df = temp_df.dropna(how='any')


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
# Copy my optimun settings for vacation destination to required DataFrame
hotel_df = temp_df

#Insert new column for hotel name
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
74,Kunda,IN,25.72,81.52,83.88,26,0,7.96,1588974153,
121,Saint George,US,37.10,-113.58,82.40,5,1,3.69,1588973982,
151,Portland,US,45.52,-122.68,81.00,24,1,19.46,1588974101,
254,Esna,EG,25.29,32.55,82.40,24,0,2.24,1588974200,
293,Bam,IR,29.11,58.36,77.59,18,0,3.71,1588974210,
304,Biaora,IN,23.87,76.92,83.95,17,0,9.80,1588974212,
448,Agadez,NE,19.75,10.25,83.05,11,0,7.45,1588974250,
538,Silver City,US,32.77,-108.28,77.00,13,1,6.93,1588974272,
542,Rāmhormoz,IR,31.28,49.60,79.68,21,0,3.85,1588974261,


In [49]:
# set params for hotel search
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
   
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    params["location"] = f"{lat},{lng}"
    
  
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
   
    hotel_name = requests.get(base_url, params=params)
    
    
    hotel_name = hotel_name.json()
    
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("No Hotels Found, Skipping...")
        
      
       


No Hotels Found, Skipping...


In [51]:
# patch-work
temp2 = hotel_df.replace('', np.nan)
temp3 = temp2.dropna(how="any")
hotel_df = temp3

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
74,Kunda,IN,25.72,81.52,83.88,26,0,7.96,1588974153,Jaishwaal Guest House
121,Saint George,US,37.10,-113.58,82.40,5,1,3.69,1588973982,Hilton Garden Inn St. George
151,Portland,US,45.52,-122.68,81.00,24,1,19.46,1588974101,"The Porter Portland, Curio Collection by Hilton"
254,Esna,EG,25.29,32.55,82.40,24,0,2.24,1588974200,Sandal Ankh1
293,Bam,IR,29.11,58.36,77.59,18,0,3.71,1588974210,Parsian Bam Hotel
304,Biaora,IN,23.87,76.92,83.95,17,0,9.80,1588974212,"MPT Tourist Motel, Biaora"
538,Silver City,US,32.77,-108.28,77.00,13,1,6.93,1588974272,Holiday Inn Express Silver City
542,Rāmhormoz,IR,31.28,49.60,79.68,21,0,3.85,1588974261,مجتمع رفاهی شهید رجایی


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))